<h1>Imports</h1>

In [1]:
# Importing necessary packages

import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

<h1>Supplementary Functions</h1>

In [2]:
# Calculating angle between joints

def calculate_angle(point1, point2, point3):
    point1 = np.array(point1)
    point3 = np.array(point3)
    point2 = np.array(point2)
    
    radians = np.arctan2(point3[1] - point2[1], point3[0] - point2[0]) - np.arctan2(point1[1] - point2[1], point1[0] - point2[0])
    angle = np.abs((radians * 180.0)/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

<h2>Analyzing Trainer footage to set state thresholds based on Shoulder angles</h2>

In [3]:
state1_angle_threshold = [0, 0]
state2_angle_threshold = [0, 0]
state3_angle_threshold = [0, 0]

In [4]:
trainer_footage = "sample_videos/Abdullah_Umar.mp4"
client_footage = './sample_videos/Abdul_Rehman.mp4'

In [5]:
cap = cv2.VideoCapture(trainer_footage)

shoulder_angles = []

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = pose.process(image)
        image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Right Keypoints
            
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
            
            
            # Left Keypoints
            
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
            
                
            
                
                
            # Shoulder Angle Calculation
            
            right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
            shoulder_angles.append(right_shoulder_angle)
            
            left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
            shoulder_angles.append(left_shoulder_angle)
        
        
        except:
            pass 
        
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        cv2.imshow('Raw Webcam Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()
    
        
# print min and max angles for elbow

print("Shoulder Angle: ", min(shoulder_angles), max(shoulder_angles))

# Set the threshold values for each state

state1_angle_threshold[0] = min(shoulder_angles)
state3_angle_threshold[1] = max(shoulder_angles)

mid_angle = round((state3_angle_threshold[1] + state1_angle_threshold[0])/2.0, 1)

state_mid_1 = round((state1_angle_threshold[0] + mid_angle)/2.0, 1)
state_mid_2 = round((mid_angle + state3_angle_threshold[1])/2.0, 1)

state1_angle_threshold[1] = state_mid_1
state2_angle_threshold[0] = state_mid_1
state2_angle_threshold[1] = state_mid_2
state3_angle_threshold[0] = state_mid_2

Shoulder Angle:  9.38574361264894 133.8707766037699


In [6]:
# print the threshold values for each state

print("State 1: ", state1_angle_threshold)
print("State 2: ", state2_angle_threshold)
print("State 3: ", state3_angle_threshold)

State 1:  [9.38574361264894, 40.5]
State 2:  [40.5, 102.7]
State 3:  [102.7, 133.8707766037699]


<p>Using shoulder angles will be better to transition through states.</p>
<p>This is because shoulders dictate the flow of our motion in this exercise most</p>

<h2>Getting Angles from Trainer Footage</h2>

In [7]:
# Right Side Angles

trainer_elbow_angle_state1_r = []
trainer_shoulder_angle_state1_r = []
trainer_hip_angle_state1_r = []
trainer_knee_angle_state1_r = []

trainer_elbow_angle_state2_r = []
trainer_shoulder_angle_state2_r = []
trainer_hip_angle_state2_r = []
trainer_knee_angle_state2_r= []

trainer_elbow_angle_state3_r = []
trainer_shoulder_angle_state3_r = []
trainer_hip_angle_state3_r = []
trainer_knee_angle_state3_r = []


# Left Side Angles

trainer_elbow_angle_state1_l = []
trainer_shoulder_angle_state1_l = []
trainer_hip_angle_state1_l = []
trainer_knee_angle_state1_l = []

trainer_elbow_angle_state2_l = []
trainer_shoulder_angle_state2_l = []
trainer_hip_angle_state2_l = []
trainer_knee_angle_state2_l= []

trainer_elbow_angle_state3_l = []
trainer_shoulder_angle_state3_l = []
trainer_hip_angle_state3_l = []
trainer_knee_angle_state3_l = []

In [8]:
cap = cv2.VideoCapture(trainer_footage)

r_state = 0
l_state = 0

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = pose.process(image)
        image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            # For Right side Points Visibility
            
            right_elbow_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility
            right_shoulder_vis = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
            right_hip_vis = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility
            right_knee_vis = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility
            right_wrist_vis = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
            right_ankle_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
            
            right_vis_sum = (right_hip_vis + right_knee_vis + right_ankle_vis + right_elbow_vis + right_wrist_vis + right_shoulder_vis)
            
            # For Left side Points Visibility
            
            left_elbow_vis = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
            left_shoulder_vis = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
            left_hip_vis = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility
            left_knee_vis = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
            left_wrist_vis = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
            left_ankle_vis = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
            
            left_vis_sum = (left_hip_vis + left_knee_vis + left_ankle_vis + left_elbow_vis + left_wrist_vis + left_shoulder_vis) 
            
            # Right Keypoints
            
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
            
            
            # Left Keypoints
            
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
            
                
            
            # Shoulder Angle Calculation and state selection
            
            left_shoulder_angle = 0
            right_shoulder_angle = 0
            
            
            right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)        
            left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
            
            
            if right_shoulder_angle >= state1_angle_threshold[0] and right_shoulder_angle <= state1_angle_threshold[1]:
                r_state = 1
            elif right_shoulder_angle >= state2_angle_threshold[0] and right_shoulder_angle <= state2_angle_threshold[1]:
                r_state = 2
            elif right_shoulder_angle >= state3_angle_threshold[0] and right_shoulder_angle <= state3_angle_threshold[1]:
                r_state = 3
            else:
                r_state = 0
            
            if left_shoulder_angle >= state1_angle_threshold[0] and left_shoulder_angle <= state1_angle_threshold[1]:
                l_state = 1
            elif left_shoulder_angle >= state2_angle_threshold[0] and left_shoulder_angle <= state2_angle_threshold[1]:
                l_state = 2
            elif left_shoulder_angle >= state3_angle_threshold[0] and left_shoulder_angle <= state3_angle_threshold[1]:
                l_state = 3
            else:
                l_state = 0
            
            
            if r_state == 1:
                trainer_shoulder_angle_state1_r.append(right_shoulder_angle.round(1))
            elif r_state == 2:
                trainer_shoulder_angle_state2_r.append(right_shoulder_angle.round(1))
            elif r_state == 3:
                trainer_shoulder_angle_state3_r.append(right_shoulder_angle.round(1)) 
            
            if l_state == 1:
                trainer_shoulder_angle_state1_l.append(left_shoulder_angle.round(1))
            elif l_state == 2:
                trainer_shoulder_angle_state2_l.append(left_shoulder_angle.round(1))
            elif l_state == 3:
                trainer_shoulder_angle_state3_l.append(left_shoulder_angle.round(1)) 
                    
                
            # Elbow Angle Calculation
            
            left_elbow_angle = 0
            right_elbow_angle = 0
            
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)  
            
                
            if r_state == 1:
                trainer_elbow_angle_state1_r.append(right_elbow_angle.round(1))
            elif r_state == 2:
                trainer_elbow_angle_state2_r.append(right_elbow_angle.round(1))
            elif r_state == 3:
                trainer_elbow_angle_state3_r.append(right_elbow_angle.round(1))
            
            
            if l_state == 1:
                trainer_elbow_angle_state1_l.append(left_elbow_angle.round(1))
            elif l_state == 2:
                trainer_elbow_angle_state2_l.append(left_elbow_angle.round(1))
            elif l_state == 3:
                trainer_elbow_angle_state3_l.append(left_elbow_angle.round(1))
            
                
            # Hip Angle Calculation
            
            right_hip_angle = 0
            left_hip_angle = 0
            
            right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
                
            if r_state == 1:
                trainer_hip_angle_state1_r.append(right_hip_angle.round(1))
            elif r_state == 2:
                trainer_hip_angle_state2_r.append(right_hip_angle.round(1))
            elif r_state == 3:
                trainer_hip_angle_state3_r.append(right_hip_angle.round(1))
            
            if l_state == 1:
                trainer_hip_angle_state1_l.append(left_hip_angle.round(1))
            elif l_state == 2:
                trainer_hip_angle_state2_l.append(left_hip_angle.round(1))
            elif l_state == 3:
                trainer_hip_angle_state3_l.append(left_hip_angle.round(1))
            
            
            # Knee Angle Calculation
            
            right_knee_angle = 0
            left_knee_angle = 0
            
            right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
                        
            if r_state == 1:
                trainer_knee_angle_state1_r.append(right_knee_angle.round(1))
            elif r_state == 2:
                trainer_knee_angle_state2_r.append(right_knee_angle.round(1))
            elif r_state == 3:
                trainer_knee_angle_state3_r.append(right_knee_angle.round(1))
                
            if l_state == 1:
                trainer_knee_angle_state1_l.append(left_knee_angle.round(1))
            elif l_state == 2:
                trainer_knee_angle_state2_l.append(left_knee_angle.round(1))
            elif l_state == 3:
                trainer_knee_angle_state3_l.append(left_knee_angle.round(1))
                
            
        except:
            pass 
        
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        cv2.imshow('Raw Webcam Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()

In [9]:
# print all trainer angle arrays

print("===============================RIGHT SIDE=====================================\n")
print("State 1:-\n")
print("Shoulder Angle: ", trainer_shoulder_angle_state1_r)
print("Elbow Angle: ", trainer_elbow_angle_state1_r)
print("Hip Angle: ", trainer_hip_angle_state1_r)
print("Knee Angle: ", trainer_knee_angle_state1_r)

print("\nState 2:-\n")
print("Shoulder Angle: ", trainer_shoulder_angle_state2_r)
print("Elbow Angle: ", trainer_elbow_angle_state2_r)
print("Hip Angle: ", trainer_hip_angle_state2_r)
print("Knee Angle: ", trainer_knee_angle_state2_r)

print("\nState 3:-\n")
print("Shoulder Angle: ", trainer_shoulder_angle_state3_r)
print("Elbow Angle: ", trainer_elbow_angle_state3_r)
print("Hip Angle: ", trainer_hip_angle_state3_r)
print("Knee Angle: ", trainer_knee_angle_state3_r)

print("\n===============================LEFT SIDE=====================================\n")
print("State 1:-\n")
print("Shoulder Angle: ", trainer_shoulder_angle_state1_l)
print("Elbow Angle: ", trainer_elbow_angle_state1_l)
print("Hip Angle: ", trainer_hip_angle_state1_l)
print("Knee Angle: ", trainer_knee_angle_state1_l)

print("\nState 2:-\n")
print("Shoulder Angle: ", trainer_shoulder_angle_state2_l)
print("Elbow Angle: ", trainer_elbow_angle_state2_l)
print("Hip Angle: ", trainer_hip_angle_state2_l)
print("Knee Angle: ", trainer_knee_angle_state2_l)

print("\nState 3:-\n")
print("Shoulder Angle: ", trainer_shoulder_angle_state3_l)
print("Elbow Angle: ", trainer_elbow_angle_state3_l)
print("Hip Angle: ", trainer_hip_angle_state3_l)
print("Knee Angle: ", trainer_knee_angle_state3_l)

===============================RIGHT SIDE=====================================

State 1:-

Shoulder Angle:  [32.3, 32.5, 32.5, 32.4, 32.5, 32.5, 32.6, 32.7, 32.9, 33.1, 33.2, 33.3, 33.3, 33.4, 33.4, 33.5, 33.4, 33.3, 33.0, 32.2, 31.0, 28.6, 27.5, 26.3, 26.1, 27.2, 27.8, 34.5, 38.7, 23.3, 17.6, 17.5, 22.4, 27.3, 33.6, 35.5, 18.4, 9.4, 12.5, 22.2, 29.3, 36.4, 37.3, 21.3, 14.3, 12.6, 18.9, 26.3, 30.1, 37.6, 23.0, 14.6, 16.0, 22.4, 29.8, 33.3, 34.3, 16.6, 10.0, 14.1, 20.9, 27.9, 30.0, 32.2]
Elbow Angle:  [127.5, 126.6, 126.3, 126.3, 126.1, 126.0, 126.0, 125.9, 125.8, 125.7, 125.5, 125.2, 125.2, 125.1, 124.6, 124.2, 123.8, 123.4, 122.9, 123.7, 126.3, 130.4, 133.1, 137.1, 146.4, 166.2, 162.4, 152.0, 158.7, 163.1, 177.2, 178.7, 174.0, 165.4, 152.3, 167.7, 166.0, 163.5, 165.3, 161.5, 164.0, 144.7, 165.4, 166.8, 172.1, 167.8, 167.1, 165.9, 143.6, 165.8, 171.0, 176.4, 170.8, 166.0, 171.6, 142.8, 166.9, 164.2, 167.6, 167.3, 173.5, 173.0, 168.7, 167.7]
Hip Angle:  [161.2, 161.0, 160.9, 160.8, 160.

In [10]:
# print min and max angles for each joint

print("===============================RIGHT SIDE=====================================\n")
print("State 1:-\n")
print("Shoulder Angle: ", min(trainer_shoulder_angle_state1_r), max(trainer_shoulder_angle_state1_r))
print("Elbow Angle: ", min(trainer_elbow_angle_state1_r), max(trainer_elbow_angle_state1_r))
print("Hip Angle: ", min(trainer_hip_angle_state1_r), max(trainer_hip_angle_state1_r))
print("Knee Angle: ", min(trainer_knee_angle_state1_r), max(trainer_knee_angle_state1_r))

print("\nState 2:-\n")
print("Shoulder Angle: ", min(trainer_shoulder_angle_state2_r), max(trainer_shoulder_angle_state2_r))
print("Elbow Angle: ", min(trainer_elbow_angle_state2_r), max(trainer_elbow_angle_state2_r))
print("Hip Angle: ", min(trainer_hip_angle_state2_r), max(trainer_hip_angle_state2_r))
print("Knee Angle: ", min(trainer_knee_angle_state2_r), max(trainer_knee_angle_state2_r))

print("\nState 3:-\n")
print("Shoulder Angle: ", min(trainer_shoulder_angle_state3_r), max(trainer_shoulder_angle_state3_r))
print("Elbow Angle: ", min(trainer_elbow_angle_state3_r), max(trainer_elbow_angle_state3_r))
print("Hip Angle: ", min(trainer_hip_angle_state3_r), max(trainer_hip_angle_state3_r))
print("Knee Angle: ", min(trainer_knee_angle_state3_r), max(trainer_knee_angle_state3_r))

print("\n===============================LEFT SIDE=====================================\n")
print("State 1:-\n")
print("Shoulder Angle: ", min(trainer_shoulder_angle_state1_l), max(trainer_shoulder_angle_state1_l))
print("Elbow Angle: ", min(trainer_elbow_angle_state1_l), max(trainer_elbow_angle_state1_l))
print("Hip Angle: ", min(trainer_hip_angle_state1_l), max(trainer_hip_angle_state1_l))
print("Knee Angle: ", min(trainer_knee_angle_state1_l), max(trainer_knee_angle_state1_l))

print("\nState 2:-\n")
print("Shoulder Angle: ", min(trainer_shoulder_angle_state2_l), max(trainer_shoulder_angle_state2_l))
print("Elbow Angle: ", min(trainer_elbow_angle_state2_l), max(trainer_elbow_angle_state2_l))
print("Hip Angle: ", min(trainer_hip_angle_state2_l), max(trainer_hip_angle_state2_l))
print("Knee Angle: ", min(trainer_knee_angle_state2_l), max(trainer_knee_angle_state2_l))

print("\nState 3:-\n")
print("Shoulder Angle: ", min(trainer_shoulder_angle_state3_l), max(trainer_shoulder_angle_state3_l))
print("Elbow Angle: ", min(trainer_elbow_angle_state3_l), max(trainer_elbow_angle_state3_l))
print("Hip Angle: ", min(trainer_hip_angle_state3_l), max(trainer_hip_angle_state3_l))
print("Knee Angle: ", min(trainer_knee_angle_state3_l), max(trainer_knee_angle_state3_l))


# put shoulder min max in variables

trainer_shoulder_angle_state1_r_min = min(trainer_shoulder_angle_state1_r)
trainer_shoulder_angle_state1_r_max = max(trainer_shoulder_angle_state1_r)

trainer_shoulder_angle_state2_r_min = min(trainer_shoulder_angle_state2_r)
trainer_shoulder_angle_state2_r_max = max(trainer_shoulder_angle_state2_r)

trainer_shoulder_angle_state3_r_min = min(trainer_shoulder_angle_state3_r)
trainer_shoulder_angle_state3_r_max = max(trainer_shoulder_angle_state3_r)

trainer_shoulder_angle_state1_l_min = min(trainer_shoulder_angle_state1_l)
trainer_shoulder_angle_state1_l_max = max(trainer_shoulder_angle_state1_l)

trainer_shoulder_angle_state2_l_min = min(trainer_shoulder_angle_state2_l)
trainer_shoulder_angle_state2_l_max = max(trainer_shoulder_angle_state2_l)

trainer_shoulder_angle_state3_l_min = min(trainer_shoulder_angle_state3_l)
trainer_shoulder_angle_state3_l_max = max(trainer_shoulder_angle_state3_l)

# put elbow min max in variables

trainer_elbow_angle_state1_r_min = min(trainer_elbow_angle_state1_r)
trainer_elbow_angle_state1_r_max = max(trainer_elbow_angle_state1_r)

trainer_elbow_angle_state2_r_min = min(trainer_elbow_angle_state2_r)
trainer_elbow_angle_state2_r_max = max(trainer_elbow_angle_state2_r)

trainer_elbow_angle_state3_r_min = min(trainer_elbow_angle_state3_r)
trainer_elbow_angle_state3_r_max = max(trainer_elbow_angle_state3_r)

trainer_elbow_angle_state1_l_min = min(trainer_elbow_angle_state1_l)
trainer_elbow_angle_state1_l_max = max(trainer_elbow_angle_state1_l)

trainer_elbow_angle_state2_l_min = min(trainer_elbow_angle_state2_l)
trainer_elbow_angle_state2_l_max = max(trainer_elbow_angle_state2_l)

trainer_elbow_angle_state3_l_min = min(trainer_elbow_angle_state3_l)
trainer_elbow_angle_state3_l_max = max(trainer_elbow_angle_state3_l)

# put hip min max in variables

trainer_hip_angle_state1_r_min = min(trainer_hip_angle_state1_r)
trainer_hip_angle_state1_r_max = max(trainer_hip_angle_state1_r)

trainer_hip_angle_state2_r_min = min(trainer_hip_angle_state2_r)
trainer_hip_angle_state2_r_max = max(trainer_hip_angle_state2_r)

trainer_hip_angle_state3_r_min = min(trainer_hip_angle_state3_r)
trainer_hip_angle_state3_r_max = max(trainer_hip_angle_state3_r)

trainer_hip_angle_state1_l_min = min(trainer_hip_angle_state1_l)
trainer_hip_angle_state1_l_max = max(trainer_hip_angle_state1_l)

trainer_hip_angle_state2_l_min = min(trainer_hip_angle_state2_l)
trainer_hip_angle_state2_l_max = max(trainer_hip_angle_state2_l)

trainer_hip_angle_state3_l_min = min(trainer_hip_angle_state3_l)
trainer_hip_angle_state3_l_max = max(trainer_hip_angle_state3_l)

# put knee min max in variables

trainer_knee_angle_state1_r_min = min(trainer_knee_angle_state1_r)
trainer_knee_angle_state1_r_max = max(trainer_knee_angle_state1_r)

trainer_knee_angle_state2_r_min = min(trainer_knee_angle_state2_r)
trainer_knee_angle_state2_r_max = max(trainer_knee_angle_state2_r)

trainer_knee_angle_state3_r_min = min(trainer_knee_angle_state3_r)
trainer_knee_angle_state3_r_max = max(trainer_knee_angle_state3_r)

trainer_knee_angle_state1_l_min = min(trainer_knee_angle_state1_l)
trainer_knee_angle_state1_l_max = max(trainer_knee_angle_state1_l)

trainer_knee_angle_state2_l_min = min(trainer_knee_angle_state2_l)
trainer_knee_angle_state2_l_max = max(trainer_knee_angle_state2_l)

trainer_knee_angle_state3_l_min = min(trainer_knee_angle_state3_l)
trainer_knee_angle_state3_l_max = max(trainer_knee_angle_state3_l)

===============================RIGHT SIDE=====================================

State 1:-

Shoulder Angle:  9.4 38.7
Elbow Angle:  122.9 178.7
Hip Angle:  150.4 175.0
Knee Angle:  152.1 178.6

State 2:-

Shoulder Angle:  41.1 102.2
Elbow Angle:  141.8 179.0
Hip Angle:  139.9 169.6
Knee Angle:  148.6 180.0

State 3:-

Shoulder Angle:  103.8 133.9
Elbow Angle:  65.7 154.3
Hip Angle:  125.5 149.0
Knee Angle:  167.9 180.0

===============================LEFT SIDE=====================================

State 1:-

Shoulder Angle:  18.2 40.1
Elbow Angle:  122.2 179.7
Hip Angle:  152.9 179.8
Knee Angle:  164.6 179.7

State 2:-

Shoulder Angle:  41.9 102.0
Elbow Angle:  125.5 171.5
Hip Angle:  137.8 176.8
Knee Angle:  164.0 178.4

State 3:-

Shoulder Angle:  103.4 130.5
Elbow Angle:  56.8 140.6
Hip Angle:  127.3 145.7
Knee Angle:  164.6 180.0


<h1>Comparing trainer skeleton to user and evaluating</h1>

In [22]:
# Variables

leniency = 10
errors = []
error_bool = False
depth_bool = False
visibility_bool = False
shoulder_angle = 0
shoulder_angle = 0
hip_angle = 0
knee_angle = 0
reps = 0


# Test Variables

client_incorrect = []
trainer_incorrect = []

In [23]:
cap = cv2.VideoCapture(client_footage)

state = 0
states_visited = []

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = pose.process(image)
        image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            
            error_bool = False
            
            
            # For Right side Points Visibility
            
            right_elbow_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility
            right_shoulder_vis = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
            right_hip_vis = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility
            right_knee_vis = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility
            right_wrist_vis = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
            right_ankle_vis = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
            
            right_vis_sum = (right_hip_vis + right_knee_vis + right_ankle_vis + right_elbow_vis + right_wrist_vis + right_shoulder_vis)
            
            # For Left side Points Visibility
            
            left_elbow_vis = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
            left_shoulder_vis = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
            left_hip_vis = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility
            left_knee_vis = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
            left_wrist_vis = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
            left_ankle_vis = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
            
            left_vis_sum = (left_hip_vis + left_knee_vis + left_ankle_vis + left_elbow_vis + left_wrist_vis + left_shoulder_vis) 
            
            
            # Right Keypoints
            
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y
            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y
            
            # Left Keypoints
            
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y
            
            
            # Checking Visibility of Joints
            
            if abs(right_vis_sum - left_vis_sum) > 1:
                visibility_bool = True
                errors.append("Visibility of Joints not same")
                client_incorrect.append("Make sure all joints are visible, stand facing the camera")
                trainer_incorrect.append("Make sure all joints are visible, stand facing the camera")
                state = 0
            else:
                visibility_bool = False
                
            
            # Shoulder Angle Calculation
            
            r_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
            l_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
            
           
           # State Selection
           
            if (r_shoulder_angle >= state1_angle_threshold[0] and r_shoulder_angle <= state1_angle_threshold[1]) and (l_shoulder_angle >= state1_angle_threshold[0] and l_shoulder_angle <= state1_angle_threshold[1]):
                state = 1
                states_visited.append(1)
                
                if len(states_visited) == 0:
                    states_visited.append(1)
                elif states_visited.__contains__(1) and states_visited.__contains__(2) and states_visited.__contains__(3):
                    states_visited = []
                    states_visited.append(1)
                    depth_bool = False
                    reps += 1
                    print("Reps: ", reps)
                elif states_visited.__contains__(1) and states_visited.__contains__(2):
                    states_visited = []
                    errors.append("State 3 not visited, hit more depth")
                    error_bool = True
                    depth_bool = True
                    state = 0
                    client_incorrect.append("Full range of motion not performed")
                    trainer_incorrect.append("Full range of motion not performed")
                    reps += 1
                    print("Reps: ", reps)
                    
                
            elif (r_shoulder_angle >= state2_angle_threshold[0] and r_shoulder_angle <= state2_angle_threshold[1]) and (l_shoulder_angle >= state2_angle_threshold[0] and l_shoulder_angle <= state2_angle_threshold[1]):
                state = 2
                states_visited.append(2)
                
            elif (r_shoulder_angle >= state3_angle_threshold[0] and r_shoulder_angle <= state3_angle_threshold[1]) and (l_shoulder_angle >= state3_angle_threshold[0] and l_shoulder_angle <= state3_angle_threshold[1]):
                state = 3
                states_visited.append(3)
            
            elif ((r_shoulder_angle >= state1_angle_threshold[0] and r_shoulder_angle <= state1_angle_threshold[1]) and (l_shoulder_angle >= state3_angle_threshold[0] and l_shoulder_angle <= state3_angle_threshold[1])) or ((r_shoulder_angle >= state3_angle_threshold[0] and r_shoulder_angle <= state3_angle_threshold[1]) and (l_shoulder_angle >= state1_angle_threshold[0] and l_shoulder_angle <= state1_angle_threshold[1])):
                errors.append("Movement of body not in sync")
                error_bool = True
                client_incorrect.append("Move right and left sides of body in sync")
                trainer_incorrect.append("Move right and left sides of body in sync")
                state = 0
            
            else:
                state = 0
                states_visited.append(0)
            
                    
            # Elbow Angle Calculation
            
            r_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            l_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
              
                
            # Hip Angle Calculation
            
            r_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            l_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)    
            
            
            # Knee Angle Calculation
            
            r_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            l_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
            
            
            # Checking Angles according to state
            
            trainer_elbow_min_r = 0
            trainer_elbow_max_r = 0
            
            trainer_shoulder_min_r = 0
            trainer_shoulder_max_r = 0
            
            trainer_hip_min_r = 0
            trainer_hip_max_r = 0
            
            trainer_knee_min_r = 0
            trainer_knee_max_r = 0
            
            trainer_elbow_min_l = 0
            trainer_elbow_max_l = 0
            
            trainer_shoulder_min_l = 0
            trainer_shoulder_max_l = 0
            
            trainer_hip_min_l = 0
            trainer_hip_max_l = 0
            
            trainer_knee_min_l = 0
            trainer_knee_max_l = 0
            
            
            if state == 1:
                trainer_elbow_min_r = trainer_elbow_angle_state1_r_min
                trainer_elbow_max_r = trainer_elbow_angle_state1_r_max
                
                trainer_shoulder_min_r = trainer_shoulder_angle_state1_r_min
                trainer_shoulder_max_r = trainer_shoulder_angle_state1_r_max
                
                trainer_hip_min_r = trainer_hip_angle_state1_r_min
                trainer_hip_max_r = trainer_hip_angle_state1_r_max
            
                trainer_knee_min_r = trainer_knee_angle_state1_r_min
                trainer_knee_max_r = trainer_knee_angle_state1_r_max
                
                trainer_elbow_min_l = trainer_elbow_angle_state1_l_min
                trainer_elbow_max_l = trainer_elbow_angle_state1_l_max
                
                trainer_shoulder_min_l = trainer_shoulder_angle_state1_l_min
                trainer_shoulder_max_l = trainer_shoulder_angle_state1_l_max
                
                trainer_hip_min_l = trainer_hip_angle_state1_l_min
                trainer_hip_max_l = trainer_hip_angle_state1_l_max
                
                trainer_knee_min_l = trainer_knee_angle_state1_l_min
                trainer_knee_max_l = trainer_knee_angle_state1_l_max

                
            elif state == 2:
                trainer_elbow_min_r = trainer_elbow_angle_state2_r_min
                trainer_elbow_max_r = trainer_elbow_angle_state2_r_max
                
                trainer_shoulder_min_r = trainer_shoulder_angle_state2_r_min
                trainer_shoulder_max_r = trainer_shoulder_angle_state2_r_max
                
                trainer_hip_min_r = trainer_hip_angle_state2_r_min
                trainer_hip_max_r = trainer_hip_angle_state2_r_max
                
                trainer_knee_min_r = trainer_knee_angle_state2_r_min
                trainer_knee_max_r = trainer_knee_angle_state2_r_max
                
                trainer_elbow_min_l = trainer_elbow_angle_state2_l_min
                trainer_elbow_max_l = trainer_elbow_angle_state2_l_max
                
                trainer_shoulder_min_l = trainer_shoulder_angle_state2_l_min
                trainer_shoulder_max_l = trainer_shoulder_angle_state2_l_max
                
                trainer_hip_min_l = trainer_hip_angle_state2_l_min
                trainer_hip_max_l = trainer_hip_angle_state2_l_max
                
                trainer_knee_min_l = trainer_knee_angle_state2_l_min
                trainer_knee_max_l = trainer_knee_angle_state2_l_max
                
            elif state == 3:
                trainer_elbow_min_r = trainer_elbow_angle_state3_r_min
                trainer_elbow_max_r = trainer_elbow_angle_state3_r_max
                
                trainer_shoulder_min_r = trainer_shoulder_angle_state3_r_min
                trainer_shoulder_max_r = trainer_shoulder_angle_state3_r_max
                
                trainer_hip_min_r = trainer_hip_angle_state3_r_min
                trainer_hip_max_r = trainer_hip_angle_state3_r_max
                
                trainer_knee_min_r = trainer_knee_angle_state3_r_min
                trainer_knee_max_r = trainer_knee_angle_state3_r_max
                
                trainer_elbow_min_l = trainer_elbow_angle_state3_l_min
                trainer_elbow_max_l = trainer_elbow_angle_state3_l_max
            
                trainer_shoulder_min_l = trainer_shoulder_angle_state3_l_min
                trainer_shoulder_max_l = trainer_shoulder_angle_state3_l_max
                
                trainer_hip_min_l = trainer_hip_angle_state3_l_min
                trainer_hip_max_l = trainer_hip_angle_state3_l_max
                
                trainer_knee_min_l = trainer_knee_angle_state3_l_min
                trainer_knee_max_l = trainer_knee_angle_state3_l_max
                
            else:
                trainer_elbow_min_r = 0
                trainer_elbow_max_r = 0
                
                trainer_shoulder_min_r = 0
                trainer_shoulder_max_r = 0
                
                trainer_hip_min_r = 0
                trainer_hip_max_r = 0
                
                trainer_knee_min_r = 0
                trainer_knee_max_r = 0
                
                trainer_elbow_min_l = 0
                trainer_elbow_max_l = 0
                
                trainer_shoulder_min_l = 0
                trainer_shoulder_max_l = 0
                
                trainer_hip_min_l = 0
                trainer_hip_max_l = 0
                
                trainer_knee_min_l = 0
                trainer_knee_max_l = 0
            
            
            # Perform checking of errors when user is not in state 0
            
            if state != 0: 
                
                # Right Elbow angle matching

                if r_elbow_angle < (trainer_elbow_min_r - leniency):
                    errors.append("Right Elbow Angle too low")
                    error_bool = True
                    client_incorrect.append(r_elbow_angle.round(1))
                    trainer_incorrect.append(trainer_elbow_min_r - leniency)
                elif r_elbow_angle > (trainer_elbow_max_r + leniency):
                    errors.append("Right Elbow Angle too high")
                    error_bool = True
                    client_incorrect.append(r_elbow_angle.round(1))
                    trainer_incorrect.append(trainer_elbow_max_r + leniency)
                    
                # Left Elbow angle matching
                
                if l_elbow_angle < (trainer_elbow_min_l - leniency):
                    errors.append("Left Elbow Angle too low")
                    error_bool = True
                    client_incorrect.append(l_elbow_angle.round(1))
                    trainer_incorrect.append(trainer_elbow_min_l - leniency)
                elif l_elbow_angle > (trainer_elbow_max_l + leniency):
                    errors.append("Left Elbow Angle too high")
                    error_bool = True
                    client_incorrect.append(l_elbow_angle.round(1))
                    trainer_incorrect.append(trainer_elbow_max_l + leniency)
                    
                    
                
                # Right Shoulder angle matching
                
                if r_shoulder_angle < (trainer_shoulder_min_r - leniency):
                    errors.append("Right Shoulder Angle too low")
                    error_bool = True
                    client_incorrect.append(r_shoulder_angle.round(1))
                    trainer_incorrect.append(trainer_shoulder_min_r - leniency)
                elif r_shoulder_angle > (trainer_shoulder_max_r + leniency):
                    errors.append("Right Shoulder Angle too high")
                    error_bool = True
                    client_incorrect.append(r_shoulder_angle.round(1))
                    trainer_incorrect.append(trainer_shoulder_max_r + leniency)
                    
                # Left Shoulder angle matching
                
                if l_shoulder_angle < (trainer_shoulder_min_l - leniency):
                    errors.append("Left Shoulder Angle too low")
                    error_bool = True
                    client_incorrect.append(l_shoulder_angle.round(1))
                    trainer_incorrect.append(trainer_shoulder_min_l - leniency)
                elif l_shoulder_angle > (trainer_shoulder_max_l + leniency):
                    errors.append("Left Shoulder Angle too high")
                    error_bool = True
                    client_incorrect.append(l_shoulder_angle.round(1))
                    trainer_incorrect.append(trainer_shoulder_max_l + leniency)
                    
                
                
                # Right Hip Angle Matching
                
                if r_hip_angle < (trainer_hip_min_r - leniency):
                    errors.append("Right Hip Angle too low")
                    error_bool = True
                    client_incorrect.append(r_hip_angle.round(1))
                    trainer_incorrect.append(trainer_hip_min_r - leniency)
                elif r_hip_angle > (trainer_hip_max_r + leniency):
                    errors.append("Right Hip Angle too high")
                    error_bool = True
                    client_incorrect.append(r_hip_angle.round(1))
                    trainer_incorrect.append(trainer_hip_max_r + leniency)
                    
                # Left Hip Angle Matching
                
                if l_hip_angle < (trainer_hip_min_l - leniency):
                    errors.append("Left Hip Angle too low")
                    error_bool = True
                    client_incorrect.append(l_hip_angle.round(1))
                    trainer_incorrect.append(trainer_hip_min_l - leniency)
                elif l_hip_angle > (trainer_hip_max_l + leniency):
                    errors.append("Left Hip Angle too high")
                    error_bool = True
                    client_incorrect.append(l_hip_angle.round(1))
                    trainer_incorrect.append(trainer_hip_max_l + leniency)
                    

                # Right Knee Angle Matching
                
                if r_knee_angle < (trainer_knee_min_r - leniency):
                    errors.append("Right Knee Angle too low")
                    error_bool = True
                    client_incorrect.append(r_knee_angle.round(1))
                    trainer_incorrect.append(trainer_knee_min_r - leniency)
                elif r_knee_angle > (trainer_knee_max_r + leniency):
                    errors.append("Right Knee Angle too high")
                    error_bool = True
                    client_incorrect.append(r_knee_angle.round(1))
                    trainer_incorrect.append(trainer_knee_max_r + leniency)
                    
                # Left Knee Angle Matching
                
                if l_knee_angle < (trainer_knee_min_l - leniency):
                    errors.append("Left Knee Angle too low")
                    error_bool = True
                    client_incorrect.append(l_knee_angle.round(1))
                    trainer_incorrect.append(trainer_knee_min_l - leniency)
                elif l_knee_angle > (trainer_knee_max_l + leniency):
                    errors.append("Left Knee Angle too high")
                    error_bool = True
                    client_incorrect.append(l_knee_angle.round(1))
                    trainer_incorrect.append(trainer_knee_max_l + leniency)
                    
            
        except:
            pass 
        
        if visibility_bool == True:
            cv2.putText(image, "Visibility Error", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                        , mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                        , mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)
                                      )
        
        if depth_bool == True:
            cv2.putText(image, "Increase ROM", (500, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                        , mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                        , mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)
                                      )
        else:
            cv2.putText(image, "ROM Perfect", (500, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                        , mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                        , mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
                                      )
            
                    
        if error_bool == True:
            cv2.putText(image, "Error", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                        , mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                        , mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)
                                      )
            
        else:
            cv2.putText(image, "No Error", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                        , mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                        , mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
                                      )
            
        
        
        cv2.imshow('Raw Webcam Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()

Reps:  1
Reps:  2
Reps:  3
Reps:  4
Reps:  5
Reps:  6


In [24]:
for i in range(len(errors)):
    print(errors[i] + " => Client: " + str(client_incorrect[i]) + " vs Trainer: " + str(trainer_incorrect[i]))

Right Knee Angle too low => Client: 136.7 vs Trainer: 138.6
Right Hip Angle too low => Client: 129.2 vs Trainer: 129.9
Right Knee Angle too low => Client: 126.5 vs Trainer: 138.6
Right Hip Angle too low => Client: 127.1 vs Trainer: 129.9
Right Knee Angle too low => Client: 123.5 vs Trainer: 138.6
Right Knee Angle too low => Client: 132.5 vs Trainer: 138.6
Right Hip Angle too high => Client: 179.9 vs Trainer: 179.6
Right Elbow Angle too low => Client: 125.7 vs Trainer: 131.8
Right Hip Angle too high => Client: 180.0 vs Trainer: 179.6
Left Elbow Angle too high => Client: 155.7 vs Trainer: 150.6
Right Elbow Angle too high => Client: 173.2 vs Trainer: 164.3
Left Elbow Angle too high => Client: 155.7 vs Trainer: 150.6
